In [ ]:
#| default_exp tbats

In [ ]:
#| export
import math 
import os 
import numpy as np 
import statsmodels.api as sm 
import matplotlib.pyplot as plt

from numba import njit 
from numpy import linalg as LA 
from scipy.stats import norm 
from scipy.optimize import minimize_scalar, fmin
from time import time 

In [ ]:
#| hide
from statsforecast.utils import (
    _quantiles, 
    _calculate_intervals, 
    _calculate_sigma
)
from statsforecast.models import AutoARIMA, ARIMA
from statsforecast.arima import arima_string 

In [ ]:
#| export
# Global variables 
NONE = float("NaN")
NOGIL = os.environ.get('NUMBA_RELEASE_GIL', 'False').lower() in ['true']
CACHE = os.environ.get('NUMBA_CACHE', 'False').lower() in ['true']

# BATS & TBATS 

In [ ]:
#| hide
# load data 

# AirPassengers 
from statsforecast.utils import AirPassengers as ap 

# USAccDeaths
USAccDeaths = np.array([9007, 8106, 8928, 9137, 10017, 10826, 11317, 10744, 9713, 9938, 9161, 8927, 
                        7750, 6981, 8038, 8422, 8714, 9512, 10120, 9823, 8743, 9129, 8710, 8680, 
                        8162, 7306, 8124, 7870, 9387, 9556, 10093, 9620, 8285, 8466, 8160, 8034, 
                        7717, 7461, 7767, 7925, 8623, 8945, 10078, 9179, 8037, 8488, 7874, 8647, 
                        7792, 6957, 7726, 8106, 8890, 9299, 10625, 9302, 8314, 8850, 8265, 8796,
                        7836, 6892, 7791, 8192, 9115, 9434, 10484, 9827, 9110, 9070, 8633, 9240]) 

# Electricity consumption 
import pandas as pd 
elec_df = pd.read_csv('PJMW_hourly.csv')
elec = elec_df['PJMW_MW']
elec = elec[0:(24*7*2+1)]
elec = np.array(elec)

# Calls dataset 
calls_df = pd.read_csv('calls_data.csv')
calls = calls_df['calls']
calls = np.array(calls) 

## Box-Cox transformation 

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def guer_cv(lam, x, season_length): 
    
    """
    Minimize this funtion to find the optimal parameter for the Box-Cox transformation.     
    """
    period = np.append(season_length, 2)
    period = np.round(np.max(period)) 
    n = len(x) 
    nyears = int(np.floor(n/period))
    nobs = np.floor(nyears*period)
    m = int(n-nobs)
    xmat = x[m:n].reshape((nyears, period))

    xmean = np.full(xmat.shape[0], fill_value = np.nan)
    for k in range(xmat.shape[0]): 
        xmean[k] = np.nanmean(xmat[k])

    xsd = np.full(xmat.shape[0], fill_value = np.nan)
    for k in range(xmat.shape[0]): 
        vals = xmat[k]
        svar = (vals-np.nanmean(vals))**2 
        svar = np.sum(svar)/(len(svar)-1) # sample variance 
        xsd[k] = np.sqrt(svar) # sample standard deviation 

    xrat = xsd/(xmean**(1-lam))

    sd = (xrat-np.nanmean(xrat))**2 # standard deviation 
    sd = np.nansum(sd)/(len(sd)-1)
    sd = np.sqrt(sd)
    
    return sd/np.nanmean(xrat)

In [ ]:
#| exporti
def guerrero(x, season_length, lower=-1, upper=2): 
    
    """Finds optimal paramater for Box-Cox transformation using Guerrero's method"""
    
    if np.any(x < 0): 
        raise ValueError("Guerrero's method for selecting a Box-Cox parameter (lambda) is for strictly positive data")
    
    max_freq = np.max(season_length)
    if len(x) <= 2*max_freq: 
        res = np.array([1]) 
    else: 
        lam = 0 # initial guess 
        opt = minimize_scalar(guer_cv, lam, args = (x, season_length), method = 'bounded', bounds = (lower, upper))
        res = np.array([opt.x])
    
    return res

In [ ]:
#| hide 
guerrero(ap, 12) # Result in R: -0.2947156

In [ ]:
#| hide 
guerrero(elec, np.array([24, 24*7])) # Result in R: 0.2943815 

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def BoxCox(y, season_length, BoxCox_lambda): 
    
    """Applies Box-Cox transformation with parameter BoxCox_lambda"""
    
    if BoxCox_lambda == 0: 
        w = np.log(y)
    else: 
        w = np.sign(y)*((np.abs(y)**BoxCox_lambda)-1)
        w = w/BoxCox_lambda
        
    return w

In [ ]:
#| hide
BoxCox_lambda = guerrero(ap, 12, lower=0, upper=1.5)
w = BoxCox(ap, 12, BoxCox_lambda) 
w[0:5]

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def InvBoxCox(w, season_length, BoxCox_lambda): 
    
    """Inverts Box-Cox transformation with parameter BoxCox_lambda"""
    
    if BoxCox_lambda == 0: 
        y = np.exp(w) 
    else: 
        sign = np.sign(BoxCox_lambda*w+1)
        y = np.abs(BoxCox_lambda*w+1)**(1/BoxCox_lambda)
        y = sign*y 
        
    return y 

In [ ]:
#| hide
yy = InvBoxCox(w, 12, BoxCox_lambda) 
yy[0:5]

## BATS functions 

Functions for BATS: 
- createXVector (numba)
- createWmatrix (numba) 
- updateWtransposeMatrix (numba)  
- createGMatrix (numba) 
- updateGMatrix (numba)
- createFMatrix (numba) 
- updateFMatrix (No - Initial fix crashed when using combination #2)
- calcModel (No)
- cutW (numba) 
- calcSeasonalSeeds (numba) 
- calcBATSFaster (No)
- checkAdmissibility (No - Computes roots of polynomial and eigenvalues)
- set_arma_errors (numba)
- extract_params (numba) 
- calcLikelihood (No - Calls checkAmissibility)
- utils (numba) 

### createXVector

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def createXVector(b, s_vector, d_vector, epsilon_vector): 

    l = 0.0
    x = np.array([l])
    if b is not None: 
        x = np.append(x, b)
    if s_vector is not None: 
        x = np.concatenate((x, s_vector))
    if d_vector is not None: 
        x = np.concatenate((x, d_vector)) 
    if epsilon_vector is not None: 
        x = np.concatenate((x, epsilon_vector))
    
    return x 

### createWMatrix

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def createWMatrix(phi, seasonal_periods, ar_coeffs, ma_coeffs): 
    
    numCols = 1 
    adjustPhi = 0
    length_seasonal = 0

    if phi is not None: 
        adjustPhi = 1
        numCols = numCols+1 
    if seasonal_periods is not None: 
        for k in range(len(seasonal_periods)): 
            length_seasonal = length_seasonal+seasonal_periods[k]
        numCols = numCols+length_seasonal
    if ar_coeffs is not None:
        p = len(ar_coeffs)
        numCols = numCols+p
    if ma_coeffs is not None: 
        q = len(ma_coeffs) 
        numCols = numCols+q

    w_transpose = np.zeros((1, numCols))

    if seasonal_periods is not None: 
        position = adjustPhi 
        for k in range(len(seasonal_periods)): 
            position = position + seasonal_periods[k]
            w_transpose[0,position] = 1.0 

    w_transpose[0,0] = 1.0 

    if phi is not None: 
        w_transpose[0,1] = phi

    if ar_coeffs is not None:
        for k in range(p): 
            w_transpose[0,(adjustPhi+length_seasonal+k+1)] = ar_coeffs[k]

    if ma_coeffs is not None: 
        for k in range(q): 
            w_transpose[0,(adjustPhi+length_seasonal+p+k+1)] = ma_coeffs[k]
    
    w = np.transpose(w_transpose)
            
    return w_transpose, w 

### updateWTransposeMatrix

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def updateWtransposeMatrix(w_transpose, phi, tau, ar_coeffs, ma_coeffs):
    p, q = findPQ(ar_coeffs, ma_coeffs) 
    adjBeta = 0 
    if phi is not None: 
        adjBeta = 1 
        w_transpose[0,1] = phi 
    if p > 0: 
        if ar_coeffs is not None:
            for i in range(1,p+1): 
                w_transpose[0, adjBeta+tau+i] = ar_coeffs[i-1]
        if q > 0: 
            if ma_coeffs is not None: 
                for i in range(1,q+1): 
                    w_transpose[0, adjBeta+tau+p+i] = ma_coeffs[i-1]
    elif q > 0: 
        if ma_coeffs is not None: 
            for i in range(1,q+1): 
                w_transpose[0, adjBeta+tau+i] = ma_coeffs[i-1]
    return w_transpose

### createGMatrix

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def createGMatrix(alpha, beta, gamma_v, seasonal_periods, ar_coeffs, ma_coeffs): 
    numCols = 1 
    adjustBeta = 0 

    if ar_coeffs is not None: 
        p = len(ar_coeffs) 
        numCols = numCols+p 
    if ma_coeffs is not None: 
        q = len(ma_coeffs) 
        numCols = numCols+q
    if beta is not None: 
        numCols = numCols+1
        adjustBeta = 1 

    gamma_length = 0 
    if gamma_v is not None and seasonal_periods is not None: 
        for k in range(len(seasonal_periods)): 
            gamma_length = gamma_length+seasonal_periods[k]
        numCols = numCols+gamma_length 
    
    g_transpose = np.zeros((1,numCols))
    g_transpose[0,0] = alpha 
    if beta is not None: 
        g_transpose[0,1] = beta 
    if gamma_v is not None and seasonal_periods is not None: 
        position = adjustBeta+1
        g_transpose[0, position] = gamma_v[0]
        if len(gamma_v) > 1: 
            for k in range(len(gamma_v)-1): 
                position = position+seasonal_periods[k]
                g_transpose[0, position] = gamma_v[k+1]
    if ar_coeffs is not None: 
        g_transpose[0, (adjustBeta+gamma_length+1)] = 1 
    if ma_coeffs is not None: 
        g_transpose[0, (adjustBeta+gamma_length+len(ar_coeffs)+1)] = 1 
        
    gamma_bold_matrix = g_transpose[0, (1+adjustBeta):(adjustBeta+gamma_length+1)]
    
    g = np.transpose(g_transpose)

    return g_transpose, g, gamma_bold_matrix 

### updateGMatrix

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def updateGMatrix(g, gamma_bold_matrix, alpha, beta, gamma_v, seasonal_periods):
    adjBeta = 0 
    g[0,0] = alpha 
    if beta is not None: 
        g[1,0] = beta
        adjBeta = 1
    if gamma_bold_matrix is not None and seasonal_periods is not None: 
        position = adjBeta+1
        bPos = 0 
        gamma_bold_matrix[bPos] = gamma_v[0]
        g[position, 0] = gamma_v[0]
        if len(gamma_v) > 1: 
            for s in range(len(seasonal_periods)-1): 
                position = position+seasonal_periods[s]
                bPos = bPos+seasonal_periods[s] 
                g[position, 0] = gamma_v[(s+1)]
    return g, gamma_bold_matrix 

### createFMatrix

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def createFMatrix(alpha, beta, phi, seasonal_periods, gamma_bold_matrix, ar_coeffs, ma_coeffs): 
    # 1. Alpha row 
    F = np.array([[1.0]])
    if beta is not None:
        F = np.concatenate((F, np.array([phi]).reshape(1,1)), axis = 1)
    if seasonal_periods is not None: 
        tau = np.nansum(seasonal_periods) 
        zero_tau = np.zeros(tau).reshape(1, tau)
        F = np.concatenate((F, zero_tau), axis = 1) 
    if ar_coeffs is not None: 
        p = len(ar_coeffs) 
        alpha_phi = alpha*ar_coeffs
        alpha_phi = alpha_phi.reshape(F.shape[0], alpha_phi.shape[0]) 
        F = np.concatenate((F, alpha_phi), axis = 1)
    if ma_coeffs is not None: 
        q = len(ma_coeffs) 
        alpha_theta = alpha*ma_coeffs
        alpha_theta = alpha_theta.reshape(F.shape[0], alpha_theta.shape[0]) 
        F = np.concatenate((F, alpha_theta), axis = 1)

    # 2. Beta row 
    if beta is not None: 
        beta_row = np.array([[0, phi]]) 
        if seasonal_periods is not None: 
            beta_row = np.concatenate((beta_row, zero_tau), axis = 1)
        if ar_coeffs is not None: 
            beta_phi = beta*ar_coeffs 
            beta_phi = beta_phi.reshape(beta_row.shape[0], beta_phi.shape[0])
            beta_row = np.concatenate((beta_row, beta_phi), axis = 1) 
        if ma_coeffs is not None: 
            beta_theta = beta*ma_coeffs 
            beta_theta = beta_theta.reshape(beta_row.shape[0], beta_theta.shape[0]) 
            beta_row = np.concatenate((beta_row, beta_theta), axis = 1)
        F = np.concatenate((F, beta_row), axis = 0)
    
    # 3. Seasonal row 
    if seasonal_periods is not None: 
        seasonal_row = np.transpose(zero_tau)
        if beta is not None: 
            seasonal_row = np.concatenate((seasonal_row, seasonal_row), axis = 1)

        # Make A matrix
        for k in seasonal_periods: 
            if k == seasonal_periods[0]: 
                a_row_one = np.zeros((1, k))
                a_row_one[0,(k-1)] = 1 
                a_row_two = np.concatenate((np.eye(k-1), np.zeros((k-1,1))), axis = 1)
                A = np.concatenate((a_row_one, a_row_two), axis = 0)
            else: 
                # This is the case with multiple seasonalities
                old_A_rows = A.shape[0] 
                old_A_columns = A.shape[1]
                a_row_one = np.zeros((1, k))
                a_row_one[0,(k-1)] = 1 
                a_row_two = np.concatenate((np.eye(k-1), np.zeros((k-1,1))), axis = 1)
                Ak = np.concatenate((a_row_one, a_row_two), axis = 0) 
                A = np.concatenate((A, np.zeros((Ak.shape[0], old_A_columns))), axis = 0) 
                A = np.concatenate((A, np.zeros((A.shape[0], Ak.shape[1]))), axis = 1) 
                A[(old_A_rows):(old_A_rows+Ak.shape[0]), (old_A_columns):(old_A_columns+Ak.shape[1])] = Ak
                
        seasonal_row = np.concatenate((seasonal_row, A), axis = 1)

        if ar_coeffs is not None: 
            B = np.outer(gamma_bold_matrix, ar_coeffs)
            seasonal_row = np.concatenate((seasonal_row, B), axis = 1)
        if ma_coeffs is not None: 
            C = np.outer(gamma_bold_matrix, ma_coeffs) 
            seasonal_row = np.concatenate((seasonal_row, C), axis = 1) 

        F = np.concatenate((F, seasonal_row), axis = 0)

    # 4. AR rows 
    if ar_coeffs is not None: 
        p = len(ar_coeffs) 
        ar_rows = np.zeros((p, 1)) 
        if beta is not None: 
            ar_rows = np.concatenate((ar_rows, ar_rows), axis = 1) 
        if seasonal_periods is not None: 
            ar_seasonal_zeros = np.zeros((p, tau))
            ar_rows = np.concatenate((ar_rows, ar_seasonal_zeros), axis = 1) 
        ident = np.eye(p-1) 
        ident = np.concatenate((ident, np.zeros((p-1, 1))), axis = 1) 
        ar_part = np.concatenate((ar_coeffs.reshape(1, ar_coeffs.shape[0]), ident), axis = 0)
        ar_rows = np.concatenate((ar_rows, ar_part), axis = 1)
        if ma_coeffs is not None: 
            q = len(ma_coeffs)
            ma_in_ar = np.zeros((p, q)) 
            ma_in_ar[0,:] = ma_coeffs 
            ar_rows = np.concatenate((ar_rows, ma_in_ar), axis = 1)
        F = np.concatenate((F, ar_rows), axis = 0)
        
    # 5. MA rows 
    if ma_coeffs is not None: 
        q = len(ma_coeffs) 
        ma_rows = np.zeros((q, 1)) 
        if beta is not None: 
            ma_rows = np.concatenate((ma_rows, ma_rows), axis = 1)
        if seasonal_periods is not None: 
            ma_seasonal = np.zeros((q, tau))
            ma_rows = np.concatenate((ma_rows, ma_seasonal), axis = 1) 
        if ar_coeffs is not None: 
            ar_in_ma = np.zeros((q, p)) 
            ma_rows = np.concatenate((ma_rows, ar_in_ma), axis = 1) 
        ident = np.eye(q-1)
        ident = np.concatenate((ident, np.zeros((q-1, 1))), axis = 1) 
        ma_part = np.concatenate((np.zeros((1,q)), ident), axis = 0) 
        ma_rows = np.concatenate((ma_rows, ma_part), axis = 1) 
        F = np.concatenate((F, ma_rows), axis = 0)
            
    return F     

### updateFMatrix 

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def updateFMatrix(F, phi, alpha, beta, gamma_bold_matrix, ar_coeffs, ma_coeffs, tau):
    if beta is not None: 
        F[0,1] = phi 
        F[1,1] = phi 
        betaAdjust = 1
    else: 
        betaAdjust = 0 
    p, q = findPQ(ar_coeffs, ma_coeffs)
    if ar_coeffs is not None: 
        F[0, (betaAdjust+tau+1):(betaAdjust+tau+p+1)] = alpha*ar_coeffs
        if betaAdjust == 1:  
            F[1, (betaAdjust+tau+1):(betaAdjust+tau+p+1)] = beta*ar_coeffs 
        if tau > 0: 
            B = np.dot(gamma_bold_matrix.reshape((gamma_bold_matrix.shape[0],1)), ar_coeffs.reshape((1,ar_coeffs.shape[0])))
            F[(1+betaAdjust):(betaAdjust+tau+1), (betaAdjust+tau+1):(betaAdjust+tau+p+1)] = B 
        F[betaAdjust+tau+1,(betaAdjust+tau+1):(betaAdjust+tau+p+1)] = ar_coeffs 
    if ma_coeffs is not None: 
        F[0, (betaAdjust+tau+p+1):(betaAdjust+tau+p+q+1)] = alpha*ma_coeffs 
        if betaAdjust == 1:  
            F[1, (betaAdjust+tau+p+1):(betaAdjust+tau+p+q+1)] = beta*ma_coeffs
        if tau > 0: 
            C = np.dot(gamma_bold_matrix.reshape((gamma_bold_matrix.shape[0],1)),ma_coeffs.reshape((1,ma_coeffs.shape[0]))) 
            F[(1+betaAdjust):(betaAdjust+tau+1), (betaAdjust+tau+p+1):(betaAdjust+tau+p+q+1)] = C 
        F[betaAdjust+tau+1,(betaAdjust+tau+p+1):(betaAdjust+tau+p+q+1)] = ma_coeffs 

    return F 

### calcModel

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def calcModel(data, x_nought, F, g, w, w_transpose): 
    n = len(data) 
    dimF = F.shape[0]
    x = np.zeros((len(x_nought), n))
    y_hat = np.zeros((1, n))
    e = np.zeros((1, n))
    y_hat[0,0] = np.dot(w_transpose, x_nought)
    e[0,0] = data[0]-y_hat[0,0]
    x[:,0] = np.dot(F, x_nought)+np.transpose(g*e[0,0])
    data = data.reshape((1,n))

    for k in range(1,n): 
        y_hat[0,k] = np.dot(w_transpose, x[:,k-1])
        e[0,k] = data[0,k]-y_hat[0,k]
        x[:,k] = (np.dot(F,x[:,k-1]).reshape((dimF,1))+g*e[0,k]).reshape((dimF,))
        
    return e, y_hat, x

### cutW

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def cutW(seasonal_periods, beta, w_tilda_transpose, ar_coeffs, ma_coeffs): 
    mask = np.zeros((len(seasonal_periods),), dtype = 'int')
    n = len(seasonal_periods)

    while n > 1: 
        for k in range(n-1):
            if (seasonal_periods[n-1] % seasonal_periods[k] == 0): 
                mask[k] = 1 
        n = n-1 
    if len(seasonal_periods) > 1: 
        for k in range(len(seasonal_periods), 2):
            for j in range((k-1), 1):
                hcf = findGCD(seasonal_periods[k], seasonal_periods[j]) 
                if hcf != 1: 
                    if (mask[k] != 1) and (mask[j] != 1):
                        mask[k] = hcf*(-1)

    w_pos_counter = 0 
    w_pos = 0
    
    if beta is not None: 
        w_pos = w_pos+1
    for k in range(len(seasonal_periods)): 
        cols = np.arange(0, w_tilda_transpose.shape[1], 1)
        if mask[w_pos_counter] == 1: 
            new_cols = np.delete(cols, np.arange((w_pos+1), w_pos+seasonal_periods[k]+1, 1)) 
            w_tilda_transpose = w_tilda_transpose[:, new_cols]
        elif mask[w_pos_counter] < 0: 
            w_pos = w_pos+seasonal_periods[k]
            new_cols = np.delete(cols, np.arange(w_pos, w_pos+mask[w_pos_counter]+2, 1))  
            w_tilda_transpose = w_tilda_transpose[:, new_cols]
        else: 
            w_pos = w_pos+seasonal_periods[k] 
            new_cols = np.delete(cols, w_pos)
            w_tilda_transpose = w_tilda_transpose[:, new_cols]
            w_pos = w_pos-1
        w_pos_counter = w_pos_counter+1

    p, q = findPQ(ar_coeffs, ma_coeffs)

    if (p != 0 or q != 0): 
        end_cut = w_tilda_transpose.shape[1]-1
        start_cut = end_cut-(p+q)+1
        cols = np.arange(0, w_tilda_transpose.shape[1], 1)
        new_cols = np.delete(cols, np.arange(start_cut, end_cut+1, 1)) 
        w_tilda_transpose = w_tilda_transpose[:, new_cols]

    return w_tilda_transpose, mask

### calcSeasonalSeeds

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def calcSeasonalSeeds(seasonal_periods, beta, ar_coeffs, ma_coeffs, mask, coeffs): 
    x_pos_counter = 0 
    sum_n = 0 
    if beta is not None: 
        x_pos = 1 
        new_x_nought = coeffs[0:2]
    else: 
        x_pos = 0 
        new_x_nought = coeffs[0:1] 
    new_x_nought = new_x_nought.reshape((len(new_x_nought),1))
    for k in range(len(seasonal_periods)): 
        if mask[x_pos_counter] == 1: 
            season = np.zeros((seasonal_periods[k], 1))
            new_x_nought = np.concatenate((new_x_nought, season), axis = 0)
        elif mask[x_pos_counter] < 0: 
            extract = coeffs[(x_pos+1):(x_pos+seasonal_periods[k]+mask[x_pos_counter]+1)]
            n = np.nansum(extract) 
            sum_n = sum_n + n/seasonal_periods[k] 
            current_periodicity = extract-n/seasonal_periods[k]
            current_periodicity = current_periodicity.reshape((len(current_periodicity), 1))
            additional = np.array([-n/seasonal_periods[k]])
            additional = additional.reshape((1,1))
            current_periodicity = np.concatenate((current_periodicity, additional), axis = 0)
            new_x_nought = np.concatenate((new_x_nought, current_periodicity), axis = 0) 
            x_pos = x_pos+seasonal_periods[k]+mask[x_pos_counter]
        else: 
            n = np.nansum(coeffs[(x_pos+1):(x_pos+seasonal_periods[k])])
            sum_n = sum_n+n/seasonal_periods[k]
            current_periodicity = coeffs[(x_pos+1):(x_pos+seasonal_periods[k])]-n/seasonal_periods[k]
            current_periodicity = np.concatenate((current_periodicity, np.array([-n/seasonal_periods[k]])), axis = 0)
            current_periodicity = current_periodicity.reshape((len(current_periodicity), 1))
            new_x_nought = np.concatenate((new_x_nought, current_periodicity), axis = 0)
            x_pos = x_pos+seasonal_periods[k]+1
        x_pos_counter = x_pos_counter+1 

    # Get the ARMA error seed states if they exist 
    p, q = findPQ(ar_coeffs, ma_coeffs)

    if (p != 0 or q != 0): 
        arma_seed_states = np.zeros((p+q,1))
        x_nought = np.concatenate((new_x_nought, arma_seed_states), axis = 0) 
    else: 
        x_nought = new_x_nought
        
    return x_nought 

### calcBATSFaster

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def calcBATSFaster(ys, yHats, wTransposes, Fs, gs, e, xs, xNought_s, sPeriods_s, betaV, tau, BoxCox_lambda, p, q): 
    
    lengthARMA = p+q

    if sPeriods_s is not None: 
        sPeriods = sPeriods_s # Integer 
        lengthSeasonal = len(sPeriods_s)

    if betaV is not None: 
        adjBeta = 1 
    else: 
        adjBeta = 0

    if sPeriods_s is not None: 
        # One 
        yHats[0,0] = np.dot(wTransposes[0, 0:(adjBeta+1)],xNought_s[0:(adjBeta+1),0])
        previousS = 0 
        for i in range(lengthSeasonal): 
            yHats[0,0] = yHats[0,0]+xNought_s[(previousS+sPeriods[i]+adjBeta), 0]
            previousS = previousS+sPeriods[i]
        if lengthARMA > 0: 
            yHats[:,0] = yHats[0,0]+np.dot(wTransposes[0, (tau+adjBeta+1):(xNought_s.shape[0])], xNought_s[(tau+adjBeta+1):(xNought_s.shape[0]), 0])

        # Two 
        e[0,0] = ys[0]-yHats[0,0]

        # Three 
        xs[0:(adjBeta+1),0] = np.dot(Fs[0:(adjBeta+1), 0:(adjBeta+1)], xNought_s[0:(adjBeta+1),:]).reshape((xs[0:(adjBeta+1),0].shape[0],))
        if lengthARMA > 0: 
            xs[0:(adjBeta+1),0] = xs[0:(adjBeta+1),0]+np.dot(Fs[0:(adjBeta+1),(adjBeta+tau+1):Fs.shape[1]],xNought_s[(adjBeta+tau+1):Fs.shape[1]]).reshape((xs[0:(adjBeta+1),0].shape[0],)) 
        previousS = 0
        for i in range(lengthSeasonal): 
            xs[(adjBeta+previousS+1),0] = xNought_s[(adjBeta+previousS+sPeriods[i]),0]
            if lengthARMA > 0: 
                res = 42 
            xs[(adjBeta+previousS+2):(adjBeta+previousS+sPeriods[i]+1), 0] = xNought_s[(adjBeta+previousS+1):(adjBeta+previousS+sPeriods[i]),0]
            previousS = previousS+sPeriods[i]
        if p > 0: 
            xs[(adjBeta+tau+1),0] = np.dot(Fs[(adjBeta+tau+1),(adjBeta+tau+1):(Fs.shape[1])],xNought_s[(adjBeta+tau+1):(Fs.shape[0])])
            if p > 1: 
                xs[(adjBeta+tau+2):(adjBeta+tau+p+1),0] = xNought_s[(adjBeta+tau+1):(adjBeta+tau+p)].reshape((xNought_s[(adjBeta+tau+1):(adjBeta+tau+p)].shape[0],))
        if q > 0: 
            xs[(adjBeta+tau+p+1),0] = 0
            if q > 1: 
                xs[(adjBeta+tau+p+2):(adjBeta+tau+p+q),0] = xNought_s[(adjBeta+tau+p+1):(adjBeta+tau+p+q)].reshape((xNought_s[(adjBeta+tau+p+1):(adjBeta+tau+p+q)],))

        ##### 
        xs[0,0] = xs[0,0]+gs[0,0]*e[0,0]
        if adjBeta == 1: 
            xs[1,0] = xs[1,0]+gs[1,0]*e[0,0] 
        previousS = 0
        for i in range(lengthSeasonal): 
            xs[(adjBeta+previousS+1),0] = xs[(adjBeta+previousS+1),0]+gs[(adjBeta+previousS+1),0]*e[0,0]
            previousS = previousS+sPeriods[i]
        if p > 0: 
            xs[(adjBeta+tau+1),0] = xs[(adjBeta+tau+1),0]+e[0,0] 
            if q > 0:
                xs[(adjBeta+tau+p+1),0] = xs[(adjBeta+tau+p+1),0]+e[0,0]
        elif q > 0: 
            xs[(adjBeta+tau+1),0] = xs[(adjBeta+tau+1),0]+e[0,0] 
        #####

        for t in range(1, ys.shape[0]):
            yHats[0,t] = np.dot(wTransposes[0, 0:(adjBeta+1)],xs[0:(adjBeta+1),(t-1)])
            previousS = 0 
            for i in range(lengthSeasonal): 
                yHats[0,t] = yHats[0,t]+xs[(previousS+sPeriods[i]+adjBeta),(t-1)]
                previousS = previousS+sPeriods[i] 
            if lengthARMA > 0: 
                yHats[0,t] = yHats[0,t] + np.dot(wTransposes[0,(tau+adjBeta+1):xNought_s.shape[0]],xs[(tau+adjBeta+1):xs.shape[0],(t-1)])  
            e[0,t] = ys[t]-yHats[0,t]
            xs[0:(adjBeta+1), t] = np.dot(Fs[0:(adjBeta+1),0:(adjBeta+1)],xs[0:(adjBeta+1), (t-1)])
            if lengthARMA > 0: 
                xs[0:(adjBeta+1),t] = xs[0:(adjBeta+1),t]+np.dot(Fs[0:(adjBeta+1),(adjBeta+tau+1):(Fs.shape[1])],xs[(adjBeta+tau+1):Fs.shape[1],(t-1)])
            previousS = 0
            for i in range(lengthSeasonal): 
                xs[(adjBeta+previousS+1),t] = xs[(adjBeta+previousS+sPeriods[i]),(t-1)]
                if lengthARMA > 0: 
                    xs[(adjBeta+previousS+1),t] = xs[(adjBeta+previousS+1),t]+np.dot(Fs[(adjBeta+previousS+1),(adjBeta+tau+1):(Fs.shape[1])],xs[(adjBeta+tau+1):(Fs.shape[1]),(t-q)])
                xs[(adjBeta+previousS+2):(adjBeta+previousS+sPeriods[i]+1), t] = xs[(adjBeta+previousS+1):(adjBeta+previousS+sPeriods[i]),(t-1)]
                previousS = previousS+sPeriods[i] 
            if p > 0: 
                xs[(adjBeta+tau+1),t] = np.dot(Fs[(adjBeta+tau+1), (adjBeta+tau+1):Fs.shape[1]],xs[(adjBeta+tau+1):Fs.shape[1],(t-1)]) 
                if p > 1: 
                    xs[(adjBeta+tau+2):(adjBeta+tau+p+1),t] = xs[(adjBeta+tau+1):(adjBeta+tau+p),(t-1)]
            if q > 0: 
                xs[(adjBeta+tau+p+1),t] = 0 
                if q > 1: 
                    xs[(adjBeta+tau+p+2):(adjBeta+tau+p+q+1),t] = xs[(adjBeta+tau+p+1):(adjBeta+tau+p+q),(t-1)]
            xs[0,t] = xs[0,t]+gs[0,0]*e[0,t]
            if adjBeta == 1: 
                xs[1,t] = xs[1,t]+gs[1,0]*e[0,t]
            previousS = 0 
            for i in range(lengthSeasonal): 
                xs[(adjBeta+previousS+1),t] = xs[(adjBeta+previousS+1),t]+gs[(adjBeta+previousS+1),0]*e[0,t]
                previousS = previousS+sPeriods[i] 
            if p > 0: 
                xs[(adjBeta+tau+1),t] = xs[(adjBeta+tau+1),t]+e[0,t] 
                if q > 0: 
                    xs[(adjBeta+tau+p+1),t] = xs[(adjBeta+tau+p+1),t]+e[0,t] 
            elif q > 0: 
                xs[(adjBeta+tau+1),t] = xs[(adjBeta+tau+1),t]+e[0,t]
   
    return yHats, e

### checkAdmissibility

In [ ]:
#| exporti
def checkAdmissibility(BoxCox_lambda, bc_lower_bound, bc_upper_bound, phi, ar_coeffs, ma_coeffs, D): 
    if BoxCox_lambda is not None: 
        if (BoxCox_lambda < bc_lower_bound) or (BoxCox_lambda >= bc_upper_bound): 
            return False 

    if phi is not None: 
        if (phi < 0.8) or (phi > 1): 
            return False 

    if ar_coeffs is not None:
        ar_lags = np.where(np.abs(ar_coeffs) > 1e-8) 
        ar_count = (np.abs(ar_coeffs) > 1e-8).sum()
        if  ar_count > 0: 
            pval = np.max(ar_lags) 
            poly = np.polynomial.Polynomial([1]+[-ar_coeffs[i] for i in range(pval+1)])
            roots = poly.roots()
            if min(np.abs(roots)) < 1+1e-2: 
                return False 
            
    if ma_coeffs is not None: 
        ma_lags = np.where(np.abs(ma_coeffs) > 1e-8) 
        ma_count = (np.abs(ma_coeffs) > 1e-8).sum()
        if len(ma_lags) > 0: 
            qval = np.max(ma_lags) 
            poly = np.polynomial.Polynomial([1]+[-ma_coeffs[i] for i in range(qval+1)])
            roots = poly.roots()
            if min(np.abs(roots)) < 1+1e-2: 
                return False 
  
    D_eigen_values = LA.eigvals(D)
    return np.all(abs(D_eigen_values) < 1+1e-2)

### set_arma_errors

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def set_arma_errors(ar_coeffs, ma_coeffs): 
    if len(ar_coeffs) == 0: 
        ar_coeffs = None 
    if len(ma_coeffs) == 0: 
        ma_coeffs = None
    return ar_coeffs, ma_coeffs

### extract_params 

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def extract_params(params, seasonal_periods, use_boxcox, use_trend, use_damped_trend, use_arma_errors, p, q): 
    if use_boxcox: 
        BoxCox_lambda = params[0]
        alpha = params[1]
        index = 2
    else: 
        BoxCox_lambda = None
        alpha = params[0] 
        index = 1 
        
    if use_trend and use_damped_trend and use_arma_errors:         
        beta = params[index] 
        phi = params[index+1] 
        gamma_v = params[(index+2):(index+2+len(seasonal_periods))]
        ar_coeffs = params[(index+2+len(seasonal_periods)):(index+2+len(seasonal_periods)+p)]
        ma_coeffs = params[(index+2+len(seasonal_periods)+p):(index+2+len(seasonal_periods)+p+q)]
        ar_coeffs, ma_coeffs = set_arma_errors(ar_coeffs, ma_coeffs)
    elif use_trend and use_damped_trend and (not use_arma_errors): 
        beta = params[index] 
        phi = params[index+1] 
        gamma_v = params[(index+2):(index+2+len(seasonal_periods))]
        ar_coeffs = None 
        ma_coeffs = None
    elif use_trend and (not use_damped_trend) and use_arma_errors: 
        beta = params[index] 
        phi = 1
        gamma_v = params[(index+1):(index+1+len(seasonal_periods))] 
        ar_coeffs = params[(index+1+len(seasonal_periods)):(index+1+len(seasonal_periods)+p)]
        ma_coeffs = params[(index+1+len(seasonal_periods)+p):(index+1+len(seasonal_periods)+p+q)] 
        ar_coeffs, ma_coeffs = set_arma_errors(ar_coeffs, ma_coeffs)
    elif use_trend and (not use_damped_trend) and (not use_arma_errors): 
        beta = params[index] 
        phi = 1 
        gamma_v = params[(index+1):(index+1+len(seasonal_periods))] 
        ar_coeffs = None
        ma_coeffs = None 
    elif (not use_trend) and (not use_damped_trend) and use_arma_errors: 
        beta = None
        phi = None
        gamma_v = params[index:(index+len(seasonal_periods))] 
        ar_coeffs = params[(index+len(seasonal_periods)):(index+1+len(seasonal_periods)+p)]
        ma_coeffs = params[(index+len(seasonal_periods)+p):(index+1+len(seasonal_periods)+p+q)] 
        ar_coeffs, ma_coeffs = set_arma_errors(ar_coeffs, ma_coeffs)
    elif (not use_trend) and (not use_damped_trend) and (not use_arma_errors): 
        beta = None
        phi = None
        gamma_v = params[index:(index+len(seasonal_periods))] 
        ar_coeffs = None
        ma_coeffs = None
    else: 
        raise ValueError("use_trend can't be set to False if use_damped_trend is set to True")
        
    return BoxCox_lambda, alpha, beta, phi, gamma_v, ar_coeffs, ma_coeffs 

### calcLikelihood

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def calcLikelihood(params, use_boxcox, use_trend, use_damped_trend, use_arma_errors, seasonal_periods, tau, y_trans, w_transpose, F, g, gamma_bold_matrix, x_nought, bc_lower_bound, bc_upper_bound, p, q): 
    
    n = len(y_trans)
    
    yHats = np.zeros((1,n))
    e = np.zeros((1,n))
    xs = np.zeros((len(x_nought), n))
    
    BoxCox_lambda, alpha, beta, phi, gamma_v, ar_coeffs, ma_coeffs = extract_params(params, seasonal_periods, use_boxcox, use_trend, use_damped_trend, use_arma_errors, p, q)
    
    w_transpose = updateWtransposeMatrix(w_transpose, phi, tau, ar_coeffs, ma_coeffs) 
    g, gamma_bold_matrix = updateGMatrix(g, gamma_bold_matrix, alpha, beta, gamma_v, seasonal_periods) 
    F = updateFMatrix(F, phi, alpha, beta, gamma_bold_matrix, ar_coeffs, ma_coeffs, tau) 

    yHats, es = calcBATSFaster(y_trans, yHats, w_transpose, F, g, e, xs, x_nought, seasonal_periods, beta, tau, BoxCox_lambda, p, q)
    
    if use_boxcox: 
        log_likelihood = n*np.log(np.nansum(es**2))-2*(BoxCox_lambda-1)*np.nansum(np.log(y_trans)) 
    else: 
        log_likelihood = n*np.log(np.nansum(es**2))
    
    D = F-np.dot(g, w_transpose)
    
    if checkAdmissibility(BoxCox_lambda, bc_lower_bound, bc_upper_bound, phi, ar_coeffs, ma_coeffs, D): 
        return log_likelihood 
    else: 
        return 10**20
    
    return log_likelihood  

### utils 
- findGCD
- findPQ
- _compute_sigmah 

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def findGCD(larger, smaller): 
    """
    Find greatest common denominator 
    """
    remainder = larger % smaller 
    if remainder != 0: 
        return(findGCD(smaller, remainder))
    else: 
        return(smaller) 

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def findPQ(ar_coeffs, ma_coeffs): 
    """
    Find the length of the AR coefficients (p) 
    and the length of the MA coefficients (q) 
    """
    if ar_coeffs is not None: 
        p = len(ar_coeffs) 
    else: 
        p = 0 
    if ma_coeffs is not None: 
        q = len(ma_coeffs) 
    else: 
        q = 0 
    return p, q

In [ ]:
#| exporti
def _compute_sigmah(h, obj): 
    """ 
    Computes the sigmah requiered for prediction intervals 
    """
    var_mult = np.zeros((h,)) 
    var_mult[0] = 1 
    if h > 1: 
        for j in range(1,h): 
            if j == 1: 
                f_running = np.eye(obj['F'].shape[1]) 
            else: 
                f_running = np.dot(f_running, obj['F']) 
            cj = np.dot(np.dot(obj['w_transpose'], f_running), obj['g'])
            var_mult[j] = var_mult[j-1]+cj**2
    sigma2h = obj['sigma2']*var_mult    
    sigmah = np.sqrt(sigma2h) 
    return sigmah 

## BATS model 

### starting_params

numba

In [ ]:
#| exporti
@njit(nogil=NOGIL, cache=CACHE)
def starting_params(seasonal_periods, 
                    use_trend, 
                    use_damped_trend, 
                    ar_coeffs, 
                    ma_coeffs
                   ): 
    
    if np.nansum(seasonal_periods) > 16: 
        alpha = 1e-6 
    else: 
        alpha = 0.09 

    if use_trend: 
        b = 0.0 
        if np.nansum(seasonal_periods) > 16: 
            beta = 5e-7
        else: 
            beta = 0.05 
        if use_damped_trend: 
            phi = 0.999 
        else: 
            phi = 1 
    else: 
        b = None 
        beta = None 
        phi = None 

    if seasonal_periods.size == 0: 
        gamma_v = None
        s_vector = None 
    else: 
        gamma_v = np.repeat(0.001, len(seasonal_periods))
        s_vector = np.zeros(np.nansum(seasonal_periods))

    if ar_coeffs is not None: 
        d_vector = np.zeros((len(ar_coeffs), )) 
    else: 
        d_vector = None

    if ma_coeffs is not None: 
        epsilon_vector = np.zeros((len(ma_coeffs), )) 
    else: 
        epsilon_vector = None 
        
    return alpha, b, beta, phi, gamma_v, s_vector, d_vector, epsilon_vector

### bats_model_generator

No numba support - uses fmin from scipy.optimize

In [ ]:
#| exporti
def bats_model_generator(y, seasonal_periods, use_boxcox, bc_lower_bound, bc_upper_bound, use_trend, use_damped_trend, use_arma_errors, alpha, b, beta, phi, gamma_v, s_vector, d_vector, epsilon_vector, ar_coeffs, ma_coeffs): 
    
    n = len(y)
    p, q = findPQ(ar_coeffs, ma_coeffs) 
    
    if use_boxcox: 
        BoxCox_lambda = guerrero(y, seasonal_periods, lower = bc_lower_bound, upper = bc_upper_bound) 
        y_trans = BoxCox(y, seasonal_periods, BoxCox_lambda) 

    else: 
        y_trans = y 
    
    # Set up matrices 
    x_nought = createXVector(b, s_vector, d_vector, epsilon_vector) 

    w_transpose, w = createWMatrix(phi, seasonal_periods, ar_coeffs, ma_coeffs)

    g_transpose, g, gamma_bold_matrix = createGMatrix(alpha, beta, gamma_v, seasonal_periods, ar_coeffs, ma_coeffs) 

    F = createFMatrix(alpha, beta, phi, seasonal_periods, gamma_bold_matrix, ar_coeffs, ma_coeffs)
    
    D = F-np.dot(g, w_transpose)
    
    # Find seed states 
    y_tilda, *_ = calcModel(y_trans, x_nought, F, g, w, w_transpose)

    w_tilda_transpose = np.zeros((len(y_trans), w_transpose.shape[1]))
    w_tilda_transpose[0,:] = w_transpose 
    for k in range(1, w_tilda_transpose.shape[0]): 
        w_tilda_transpose[k,:] = np.dot(w_tilda_transpose[k-1,:], D)

    if seasonal_periods is not None: 
        w_tilda_transpose, mask = cutW(seasonal_periods, beta, w_tilda_transpose, ar_coeffs, ma_coeffs)

        model = sm.OLS(y_tilda.reshape((y_tilda.shape[1],1)), w_tilda_transpose).fit()
        coeffs = model.params

        x_nought = calcSeasonalSeeds(seasonal_periods, beta, ar_coeffs, ma_coeffs, mask, coeffs)
    else: 
        if (p != 0 or q != 0):
            end_cut = w_tilda_transpose.shape[1] 
            start_cut = end_cut-(p+q)
            cols = np.arange(0, w_tilda_transpose.shape[1], 1)
            new_cols = np.delete(cols, np.arange(start_cut, end_cut, 1))
            w_tilda_transpose = w_tilda_transpose[:, new_cols]
        model = sm.OLS(y_tilda.reshape((y_tilda.shape[1],1)), w_tilda_transpose).fit()
        x_nought = model.params
        if (p != 0 or q != 0):
            arma_seed_states = np.zeros((p+q, ))
            x_nought = np.concatenate((x_nought, arma_seed_states))
        x_nought = x_nought.reshape((x_nought.shape[0], 1))

    # Minimize log-likelihood to find optimal parameters 
    if seasonal_periods is not None: 
        tau = np.nansum(seasonal_periods) 
    else: 
        tau = 0

    # Create vector with parameters 
    if use_boxcox: 
        params = np.concatenate([BoxCox_lambda, np.array([alpha])]) 
    else:  
        params = np.array([alpha])
    if beta is not None:  
        params = np.concatenate([params, np.array([beta])])
    if phi is not None: 
        params = np.concatenate([params, np.array([phi])]) 
    params = np.concatenate([params, gamma_v])
    if ar_coeffs is not None: 
        params = np.concatenate([params, ar_coeffs])
    if ma_coeffs is not None: 
        params = np.concatenate([params, ma_coeffs])

    # Solve optimization problem 
    #start = time()
    optim_params = fmin(calcLikelihood, 
                        params, 
                        args = (use_boxcox, use_trend, use_damped_trend, use_arma_errors, seasonal_periods, tau, y_trans, w_transpose, F, g, gamma_bold_matrix, x_nought, bc_lower_bound, bc_upper_bound, p, q), 
                        maxiter = 2000,
                        disp = False
                        )
    #end = time() 
    #print('Total execution time: '+str((end-start)/60)) 

    # Extract optimal parameters 
    optim_BoxCox_lambda, optim_alpha, optim_beta, optim_phi, optim_gamma_v, optim_ar_coeffs, optim_ma_coeffs = extract_params(optim_params, seasonal_periods, use_boxcox, use_trend, use_damped_trend, use_arma_errors, p, q)
    
    if use_boxcox: 
        x_nought_untransformed = InvBoxCox(x_nought, seasonal_periods, params[0])
        xx_nought = BoxCox(x_nought_untransformed, seasonal_periods, optim_BoxCox_lambda) 
        xx_nought = xx_nought.reshape((xx_nought.shape[0],))
    else: 
        xx_nought = x_nought.reshape((x_nought.shape[0],))

    # Re-set up the matrices 
    w_transpose, w = createWMatrix(optim_phi, seasonal_periods, optim_ar_coeffs, optim_ma_coeffs)

    g_transpose, g, gamma_bold_matrix = createGMatrix(optim_alpha, optim_beta, optim_gamma_v, seasonal_periods, optim_ar_coeffs, optim_ma_coeffs)

    F = createFMatrix(optim_alpha, optim_beta, optim_phi, seasonal_periods, gamma_bold_matrix, optim_ar_coeffs, optim_ma_coeffs)
    
    e, fitted, x = calcModel(y_trans, xx_nought, F, g, w, w_transpose)
    sigma2 = np.nansum((e*e))/len(y_trans) 
    
    # Calculate likelihood 
    if use_boxcox: 
        likelihood = len(y)*np.log(np.nansum(e**2))-2*(optim_BoxCox_lambda-1)*np.nansum(np.log(y)) 
    else: 
        likelihood = len(y)*np.log(np.nansum(e**2))

    # Calculate AIC
    aic = likelihood+2*(len(optim_params)+x_nought.shape[0])
    
    res = {'e': e, 
           'fitted': fitted, 
           'sigma2': sigma2, 
           'aic': aic, 
           'optim_params': optim_params, 
           'F': F, 
           'w_transpose': w_transpose, 
           'g': g, 
           'x': x
          }
    
    return res 

### bats_model 

No numba support - Returns dictionary

In [ ]:
#| exporti
def bats_model(y, 
               seasonal_periods, 
               use_boxcox,
               bc_lower_bound,
               bc_upper_bound, 
               use_trend, 
               use_damped_trend, 
               use_arma_errors,
               ): 
    
    seasonal_periods = np.sort(seasonal_periods)
    
    # Check if there are missing values 
    indices = np.where(np.isnan(y))[0]
    if len(indices) > 0: 
        max_index = indices[-1]
        y = y[max_index+1:len(y)]
    
    # Check if there are negative values 
    if np.any(y < 0): 
        use_boxcox = False 
    
    # Create first model 
    ar_coeffs = None
    ma_coeffs = None
    p, q = findPQ(ar_coeffs, ma_coeffs)
    
    # Starting parameters 
    alpha, b, beta, phi, gamma_v, s_vector, d_vector, epsilon_vector = starting_params(seasonal_periods, use_trend, use_damped_trend, ar_coeffs, ma_coeffs)
    
    model1 = bats_model_generator(y, seasonal_periods, use_boxcox, bc_lower_bound, bc_upper_bound, use_trend, use_damped_trend, use_arma_errors, alpha, b, beta, phi, gamma_v, s_vector, d_vector, epsilon_vector, ar_coeffs, ma_coeffs)
    best_model = model1  
    
    if use_arma_errors: 
        # ARMA errors from first model 
        errors = model1['e'][0]
        auto_arima = AutoARIMA(season_length = 1, d = 0)
        fit = auto_arima.fit(errors) 
        p = int(arima_string(fit.model_)[6])
        q = int(arima_string(fit.model_)[8])
        if p != 0: 
            ar_coeffs = np.zeros((p,)) 
        else: 
            ar_coeffs = None 
        if q != 0: 
            ma_coeffs = np.zeros((q,))
        else: 
            ma_coeffs = None
        
        _, _, _, _, _, s_vector, d_vector, epsilon_vector = starting_params(seasonal_periods, use_trend, use_damped_trend, ar_coeffs, ma_coeffs)
        optim_BoxCox_lambda, optim_alpha, optim_beta, optim_phi, optim_gamma_v, optim_ar_coeffs, optim_ma_coeffs = extract_params(model1['optim_params'], seasonal_periods, use_boxcox, use_trend, use_damped_trend, use_arma_errors, p, q)
        model2 = bats_model_generator(y, seasonal_periods, use_boxcox, bc_lower_bound, bc_upper_bound, use_trend, use_damped_trend, use_arma_errors, optim_alpha, b, optim_beta, optim_phi, optim_gamma_v, s_vector, d_vector, epsilon_vector, ar_coeffs, ma_coeffs)
        
        if model2['aic'] < model1['aic']: 
            best_model = model2 
            
    return best_model 

### bats_forecast 

No numba support - Returns dictionary

In [ ]:
#| exporti
def bats_forecast(mod, h, seasonal_periods): 
    
    fcst = np.zeros((h,))
    xx = np.zeros((mod['x'].shape[0], h))

    fcst[0] = np.dot(mod['w_transpose'], mod['x'][:,-1])

    xx[:,0] = np.dot(mod['F'], mod['x'][:,-1]) 

    if h > 1: 
        for t in range(1,h): 
            xx[:,t] = np.dot(mod['F'], xx[:,(t-1)])
            fcst[t] = np.dot(mod['w_transpose'], xx[:,(t-1)])
            
    return fcst

## StatsForecast pipeline 

In [ ]:
# No need to copy these functions since they are already in models. 
#| exporti
class _TS:
    def new(self):
        b = type(self).__new__(type(self))
        b.__dict__.update(self.__dict__)
        return b

In [ ]:
#| exporti
from typing import Any, Dict, List, Optional, Sequence, Tuple, Union

In [ ]:
#| exporti
def _add_fitted_pi(res, se, level):
    level = sorted(level)
    level = np.asarray(level)
    quantiles = _quantiles(level=level)
    lo = res['fitted'].reshape(-1, 1) - quantiles * se.reshape(-1, 1)
    hi = res['fitted'].reshape(-1, 1) + quantiles * se.reshape(-1, 1)
    lo = lo[:, ::-1]
    lo = {f'fitted-lo-{l}': lo[:, i] for i, l in enumerate(reversed(level))}
    hi = {f'fitted-hi-{l}': hi[:, i] for i, l in enumerate(level)}
    res = {**res, **lo, **hi}
    return res

In [ ]:
#| export
class BATS(_TS):
    """Box-Cox transform, ARMA errors, Trend, and Seasonal components model. 
    
    Parameters
    ----------
    use_boxcox : bool 
        Whether to use a BoxCox transformation 
    alias : str 
        Custom name of the model. 
    """
    def __init__(
            self, 
            seasonal_periods: Union[int, List[int]],
            use_boxcox: bool = True, 
            bc_lower_bound: Optional[int] = 0, 
            bc_upper_bound: Optional[int] = 1, 
            use_trend: bool = True, 
            use_damped_trend: bool = True,
            use_arma_errors: bool = True,
            alias: str = 'BATS'
        ):
        self.seasonal_periods = seasonal_periods  
        self.use_boxcox = use_boxcox
        self.bc_lower_bound = bc_lower_bound 
        self.bc_upper_bound = bc_upper_bound 
        self.use_trend = use_trend 
        self.use_damped_trend = use_damped_trend 
        self.use_arma_errors = use_arma_errors 
        self.alias = alias 
        
    def __repr__(self):
        return self.alias
    
    def fit(
            self,
            y: np.ndarray,
            X: Optional[np.ndarray] = None
        ):
        """Fit BATS model.

        Fit BATS model to a time series (numpy array) `y`.

        Parameters
        ----------
        y : numpy.array 
            Clean time series of shape (t, ). 

        Returns
        -------
        self : 
            BATS model.
        """
        self.model_ = bats_model(y, 
                                 seasonal_periods = self.seasonal_periods, 
                                 use_boxcox = self.use_boxcox, 
                                 bc_lower_bound = self.bc_lower_bound, 
                                 bc_upper_bound = self.bc_upper_bound,
                                 use_trend = self.use_trend, 
                                 use_damped_trend = self.use_damped_trend, 
                                 use_arma_errors = self.use_arma_errors
                                )
        return self
    
    def predict(
            self,
            h: int, 
            X: Optional[np.ndarray] = None,
            level: Optional[List[int]] = None
        ):
        """Predict with fitted BATS model.

        Parameters
        ----------
        h : int 
            Forecast horizon.
        level : List[float] 
            Confidence levels (0-100) for prediction intervals.

        Returns
        -------
        forecasts : dict 
            Dictionary with entries `mean` for point predictions and `level_*` for probabilistic predictions.
        """
        fcst = bats_forecast(self.model_, h, self.seasonal_periods)
        res = {'mean': fcst}
        if level is not None: 
            level = sorted(level) 
            quantiles = _quantiles(level)
            sigmah = _compute_sigmah(h, self.model_)
            pred_int = _calculate_intervals(res, level, h, sigmah)
            res = {**res, **pred_int}
        if self.use_boxcox: 
            res_trans = {k : InvBoxCox(v, self.seasonal_periods, self.model_['optim_params'][0]) for k, v in res.items()}
        else: 
            res_trans = res 
        
        return res_trans 
    
    def predict_in_sample(
            self, 
            level: Optional[Tuple[int]] = None
        ):
        """Access fitted BATS model predictions.

        Parameters
        ----------
        level : List[float]
            Confidence levels (0-100) for prediction intervals.

        Returns
        -------
        forecasts : dict 
            Dictionary with entries `mean` for point predictions and `level_*` for probabilistic predictions.
        """
        res = {'fitted': self.model_['fitted']}
        if level is not None:
            se = _calculate_sigma(self.model_['e'], self.model_['e'].shape[1])
            fitted_pred_int = _add_fitted_pi(res, se, level)
            res = {**res, **fitted_pred_int} 
        if self.use_boxcox: 
            res_trans = {k : InvBoxCox(v, self.seasonal_periods, self.model_['optim_params'][0]) for k, v in res.items()}
        else: 
            res_trans = res 
        
        return res_trans 
    
    def forecast(
            self,
            y: np.ndarray,
            h: int, 
            X: Optional[np.ndarray] = None,
            X_future: Optional[np.ndarray] = None,
            level: Optional[List[int]] = None,
            fitted: bool=False 
        ):
        """Memory Efficient BATS model.

        This method avoids memory burden due from object storage.
        It is analogous to `fit_predict` without storing information.
        It assumes you know the forecast horizon in advance.

        Parameters
        ----------
        y : numpy.array 
            Clean time series of shape (n, ). 
        h : int
            Forecast horizon.
        level : List[float] 
            Confidence levels (0-100) for prediction intervals.
        fitted : bool
            Whether or not returns insample predictions.

        Returns
        -------
        forecasts : dict 
            Dictionary with entries `mean` for point predictions and `level_*` for probabilistic predictions.
        """
        mod = bats_model(y, 
                         seasonal_periods = self.seasonal_periods, 
                         use_boxcox = self.use_boxcox, 
                         bc_lower_bound = self.bc_lower_bound, 
                         bc_upper_bound = self.bc_upper_bound,
                         use_trend = self.use_trend, 
                         use_damped_trend = self.use_damped_trend, 
                         use_arma_errors = self.use_arma_errors
                        )
        fcst = bats_forecast(mod, h, self.seasonal_periods)
        res = {'mean': fcst}
        if fitted: 
            res['fitted'] = mod['fitted']
        if level is not None: 
            level = sorted(level) 
            quantiles = _quantiles(level)
            sigmah = _compute_sigmah(h, mod)
            pred_int = _calculate_intervals(res, level, h, sigmah)
            res = {**res, **pred_int}
            if fitted: 
                se = _calculate_sigma(mod['e'], mod['e'].shape[1])
                fitted_pred_int = _add_fitted_pi(res, se, level)
                res = {**res, **fitted_pred_int}
                
        if self.use_boxcox: 
            res_trans = {k : InvBoxCox(v, self.seasonal_periods, mod['optim_params'][0]) for k, v in res.items()}
        else: 
            res_trans = res 
            
        return res_trans 

### Testing 

In [ ]:
#| hide
y = ap 
h = 24

In [ ]:
#| hide
bats = BATS(seasonal_periods = np.array([12]), 
            use_boxcox = True, 
            use_trend = True, 
            use_damped_trend = True,
            use_arma_errors = True
           )

In [ ]:
#| hide
# forercast 
start = time()
forecast = bats.forecast(y, h, None, None, [80,95], True)
end = time() 
print('Total execution time: '+str(end-start))

In [ ]:
#| hide
fig, ax = plt.subplots(1, 1, figsize = (20,7))
plt.plot(np.arange(0, len(y)), y, color='black')
plt.plot(np.arange(len(y), len(y) + h), forecast['mean'], label='mean', color = 'green')
plt.plot(np.arange(len(y), len(y) + h), forecast['lo-95'], label='lo-95', color = 'darkgreen')
plt.plot(np.arange(len(y), len(y) + h), forecast['hi-95'], label='hi-95', color = 'darkgreen') 
plt.fill_between(np.arange(len(y), len(y) + h), forecast['lo-95'], forecast['hi-95'], color = 'darkgreen', alpha = 0.1)
plt.plot(np.arange(0, len(y)), forecast['fitted'][0], label='fitted', color = 'royalblue')
plt.plot(np.arange(0, len(y)), forecast['fitted-lo-95'], label='fitted-lo-95', color = 'royalblue')
plt.plot(np.arange(0, len(y)), forecast['fitted-hi-95'], label='fitted-hi-95', color = 'royalblue')
plt.fill_between(np.arange(0, len(y)), forecast['fitted-lo-95'], forecast['fitted-hi-95'], color = 'royalblue', alpha = 0.1)
plt.legend()

In [ ]:
#| hide
# fit & predict 
fit = bats.fit(y) 
fcst = fit.predict(h, None, (80,95))

In [ ]:
#| hide
# predict in sample 
fitted = fit.predict_in_sample([80,95])

In [ ]:
#| hide
fig, ax = plt.subplots(1, 1, figsize = (20,7))
plt.plot(np.arange(0, len(y)), y, color='black')
plt.plot(np.arange(len(y), len(y) + h), fcst['mean'], label='mean', color = 'green')
plt.plot(np.arange(len(y), len(y) + h), fcst['lo-95'], label='lo-95', color = 'darkgreen')
plt.plot(np.arange(len(y), len(y) + h), fcst['hi-95'], label='hi-95', color = 'darkgreen') 
plt.fill_between(np.arange(len(y), len(y) + h), fcst['lo-95'], fcst['hi-95'], color = 'darkgreen', alpha = 0.1)
plt.plot(np.arange(0, len(y)), fitted['fitted'][0], label='fitted', color = 'royalblue')
plt.plot(np.arange(0, len(y)), fitted['fitted-lo-95'], label='fitted-lo-95', color = 'royalblue')
plt.plot(np.arange(0, len(y)), fitted['fitted-hi-95'], label='fitted-hi-95', color = 'royalblue')
plt.fill_between(np.arange(0, len(y)), fitted['fitted-lo-95'], fitted['fitted-hi-95'], color = 'royalblue', alpha = 0.1)
plt.legend()

### Validation 

In [ ]:
#| hide
dataset = 'AirPassengers'
#dataset = 'USAccDeaths'

In [ ]:
#| hide
if dataset == 'AirPassengers': 
    y = ap 
else: 
    y = USAccDeaths
    
seasonal_periods = np.array([12]) 
h = 24

# Results from forecast R package 
import pandas as pd 
resR = pd.read_csv('bats_R_results/bats_'+dataset+'_R.csv')
lowerR = pd.read_csv('bats_R_results/lower_'+dataset+'_R.csv')
upperR = pd.read_csv('bats_R_results/upper_'+dataset+'_R.csv')

In [ ]:
#| hide
# Electricity consumption 
# y = elec
# seasonal_periods = np.array([24, 24*7])
# h = 24*7

In [ ]:
#| hide 
# Validation 
combinations = []

for BC in [True, False]: # Possible values for use_boxcox
    for T in [[True, True],[True, False],[False, False]]: # Possible values for use_trend and use_damped_trend  
        for A in [True, False]: # Possible values for use_arma_errors [True, False]
            combinations.append((BC, T, A))

start = time()

for k in range(len(combinations)): 
    use_boxcox = combinations[k][0]
    use_trend = combinations[k][1][0]
    use_damped_trend = combinations[k][1][1] 
    use_arma_errors = combinations[k][2]
    
    bats = BATS(seasonal_periods = seasonal_periods, 
            use_boxcox = use_boxcox,
            use_trend = use_trend, 
            use_damped_trend = use_damped_trend,
            use_arma_errors = use_arma_errors,
           )
    
    # fit = bats.fit(y) 
    # fcst = fit.predict(h, None, (80,95))
    # fitted = fit.predict_in_sample([80,95])
    
    forecast = bats.forecast(y, h, None, None, [80,95], True)
    
    print('Combination '+str(k)+' validated - '+str(use_boxcox)+','+str(use_trend)+','+str(use_damped_trend)+','+str(use_arma_errors))
    
    fig, ax = plt.subplots(1, 1, figsize = (20,7))
    plt.plot(np.arange(0, len(y)), y, color = 'red')
    plt.plot(np.arange(0, len(y)), forecast['fitted'][0], label='fitted', color = 'royalblue')
    plt.plot(np.arange(0, len(y)), forecast['fitted-lo-95'], label='fitted-lo-95', color = 'royalblue')
    plt.plot(np.arange(0, len(y)), forecast['fitted-hi-95'], label='fitted-hi-95', color = 'royalblue')
    plt.fill_between(np.arange(0, len(y)), forecast['fitted-lo-95'], forecast['fitted-hi-95'], color = 'royalblue', alpha = 0.1)
    plt.plot(np.arange(len(y), len(y) + h), forecast['mean'], label='mean', color = 'green')
    plt.plot(np.arange(len(y), len(y) + h), forecast['lo-95'], label='lo-95', color = 'darkgreen')
    plt.plot(np.arange(len(y), len(y) + h), forecast['hi-95'], label='hi-95', color = 'darkgreen') 
    plt.fill_between(np.arange(len(y), len(y) + h), forecast['lo-95'], forecast['hi-95'], color = 'green', alpha = 0.1)
    plt.plot(np.arange(len(y), len(y) + h), resR.iloc[k,len(y):(len(y)+h+1)], label='R-mean', color = 'purple')
    plt.plot(np.arange(len(y), len(y) + h), lowerR.iloc[k,:], label='R-lo-95', color = 'purple')
    plt.plot(np.arange(len(y), len(y) + h), upperR.iloc[k,:], label='R-hi-95', color = 'purple')
    plt.fill_between(np.arange(len(y), len(y) + h), lowerR.iloc[k,:], upperR.iloc[k,:], color = 'purple', alpha = 0.1)
    plt.legend()
    
    # fig, ax = plt.subplots(1, 1, figsize = (20,7))
    # plt.plot(np.arange(0, len(y)), y, color = 'red')
    # plt.plot(np.arange(0, len(y)), fitted['fitted'][0], label='fitted')
    # plt.plot(np.arange(0, len(y)), fitted['fitted-lo-95'], label='fitted-lo-95', color = 'royalblue')
    # plt.plot(np.arange(0, len(y)), fitted['fitted-hi-95'], label='fitted-hi-95', color = 'royalblue')
    # plt.fill_between(np.arange(0, len(y)), fitted['fitted-lo-95'], fitted['fitted-hi-95'], color = 'royalblue', alpha = 0.1)
    # plt.plot(np.arange(len(y), len(y) + h), fcst['mean'], label='mean', color = 'green')
    # plt.plot(np.arange(len(y), len(y) + h), fcst['lo-95'], label='lo-95', color = 'darkgreen')
    # plt.plot(np.arange(len(y), len(y) + h), fcst['hi-95'], label='hi-95', color = 'darkgreen') 
    # plt.fill_between(np.arange(len(y), len(y) + h), fcst['lo-95'], fcst['hi-95'], color = 'green', alpha = 0.1)
    # plt.legend()
    
end = time() 
print('Total execution time: '+str((end-start)/60))